# PEtab import

This illustrates how the PEtab format can be used together with the ODE simulation toolbox AMICI to define ODE based parameter estimation problems for pyABC.

In [5]:
import petab
import pyabc
import amici.petab_import
from pyabc.petab import AmiciPetabImporter

In [6]:
petab_problem = petab.Problem.from_yaml("/home/yannik/benchmark-models/"
                                        "hackathon_contributions_new_data_format/"
                                        "Boehm_JProteomeRes2014/Boehm_JProteomeRes2014.yaml")
model = amici.petab_import.import_petab_problem(petab_problem)
solver = model.getSolver()

importer = AmiciPetabImporter(petab_problem, model, solver)

prior = importer.get_prior()
model = importer.get_model()
kernel = importer.get_kernel()

model(petab_problem.x_nominal_scaled)

2020-02-10 00:58:17.194 - amici.petab_import - INFO - Importing model ...
INFO:amici.petab_import:Importing model ...
2020-02-10 00:58:17.195 - amici.petab_import - INFO - Model name is 'Boehm_JProteomeRes2014'. Writing model code to '/home/yannik/pyabc/doc/examples/amici_models/Boehm_JProteomeRes2014'.
INFO:amici.petab_import:Model name is 'Boehm_JProteomeRes2014'. Writing model code to '/home/yannik/pyabc/doc/examples/amici_models/Boehm_JProteomeRes2014'.
2020-02-10 00:58:17.197 - amici.petab_import - INFO - Species: 8
INFO:amici.petab_import:Species: 8
2020-02-10 00:58:17.198 - amici.petab_import - INFO - Global parameters: 9
INFO:amici.petab_import:Global parameters: 9
2020-02-10 00:58:17.201 - amici.petab_import - INFO - Reactions: 9
INFO:amici.petab_import:Reactions: 9
2020-02-10 00:58:17.218 - amici.petab_import - INFO - Observables: 3
INFO:amici.petab_import:Observables: 3
2020-02-10 00:58:17.225 - amici.petab_import - INFO - Sigmas: 3
INFO:amici.petab_import:Sigmas: 3
2020-02-

2020-02-10 00:58:32.426 - amici.ode_export - INFO - Finished compiling cpp code                     (1.44E+01s)
INFO:amici.ode_export:Finished compiling cpp code                     (1.44E+01s)
2020-02-10 00:58:32.431 - amici.petab_import - INFO - Finished Importing PEtab model                (1.52E+01s)
INFO:amici.petab_import:Finished Importing PEtab model                (1.52E+01s)
2020-02-10 00:58:32.439 - amici.petab_import - INFO - Successfully loaded model Boehm_JProteomeRes2014 from /home/yannik/pyabc/doc/examples/amici_models/Boehm_JProteomeRes2014.
INFO:amici.petab_import:Successfully loaded model Boehm_JProteomeRes2014 from /home/yannik/pyabc/doc/examples/amici_models/Boehm_JProteomeRes2014.


running build_ext
building 'Boehm_JProteomeRes2014._Boehm_JProteomeRes2014' extension
swigging swig/Boehm_JProteomeRes2014.i to swig/Boehm_JProteomeRes2014_wrap.cpp
swig -python -c++ -modern -outdir Boehm_JProteomeRes2014 -I/home/yannik/amici/python/sdist/amici/swig -I/home/yannik/amici/python/sdist/amici/include -o swig/Boehm_JProteomeRes2014_wrap.cpp swig/Boehm_JProteomeRes2014.i
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/swig
gcc -pthread -B /home/yannik/anaconda3/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -I/home/yannik/pyabc/doc/examples/amici_models/Boehm_JProteomeRes2014 -I/home/yannik/amici/python/sdist/amici/include -I/home/yannik/amici/python/sdist/amici/ThirdParty/gsl -I/home/yannik/amici/python/sdist/amici/ThirdParty/sundials/include -I/home/yannik/amici/python/sdist/amici/ThirdParty/SuiteSparse/include -I/usr/include/hdf5/serial -I/home/yannik/anaconda3/include/python3.7m -c swig/Boehm_JP

{'llh': -138.22199570334107}

In [7]:
# this takes some time

abc = pyabc.ABCSMC(model, prior, kernel, 
                   eps=pyabc.Temperature(),
                   acceptor=pyabc.StochasticAcceptor())
abc.new(pyabc.storage.create_sqlite_db_id(), {})
abc.run()

INFO:History:Start <ABCSMC(id=20, start_time=2020-02-10 01:00:12.251740, end_time=None)>
INFO:ABC:Calibration sample before t=0.
INFO:ABC:t: 0, eps: 157108667.55297396.
Process Process-6:
Process Process-8:
Traceback (most recent call last):
Process Process-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yannik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/yannik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/yannik/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yannik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/yannik/pyabc/pyabc/sampler/multicore_evaluation_parallel.py", line 32, in work
    new_sim = simulate_one()
  File "/home/yannik/pyabc/pyabc/smc.py", line 595, in simulate_one
    

KeyboardInterrupt: 

  File "/home/yannik/pyabc/pyabc/model.py", line 121, in summary_statistics
    raw_data = self.sample(pars)
  File "/home/yannik/pyabc/pyabc/model.py", line 121, in summary_statistics
    raw_data = self.sample(pars)
  File "/home/yannik/pyabc/pyabc/smc.py", line 680, in _evaluate_proposal
    x_0)
  File "/home/yannik/pyabc/pyabc/petab.py", line 56, in model
    scaled_parameters=True)
  File "/home/yannik/pyabc/pyabc/model.py", line 270, in sample
    return self.sample_function(pars)
  File "/home/yannik/pyabc/pyabc/model.py", line 227, in accept
    sum_stats_calculator)
  File "/home/yannik/pyabc/pyabc/petab.py", line 56, in model
    scaled_parameters=True)
  File "/home/yannik/pyabc/pyabc/model.py", line 121, in summary_statistics
    raw_data = self.sample(pars)
  File "/home/yannik/pyabc/pyabc/model.py", line 270, in sample
    return self.sample_function(pars)
  File "/home/yannik/amici/python/sdist/amici/logging.py", line 144, in wrapper_timer
    rval = func(*args, **kwarg